<a href="https://colab.research.google.com/github/kitten404/my_first_LLM/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY=""
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
title = "The New generation of people in technology job market"
sample_text = ("Title: The New generation of people in technology job market "
"/n"
"Complete article: The new generation is yet being created for a new century"
)

embeddings = genai.embed_content(
    model="models/embedding-001",
    content=sample_text,
    title=title,
    task_type="RETRIEVAL_DOCUMENT"
)

print(embeddings)

{'embedding': [0.006971295, -0.014906378, -0.043062013, -0.044643104, 0.08333732, 0.03887606, 0.018828282, 0.028517088, -0.010850444, 0.09455997, -0.016874462, 0.021766715, -0.048524674, -0.019866494, 0.016588045, -0.040779058, 0.019332234, 0.053781874, -0.02781566, -0.010497367, 0.039181497, -0.03060142, -0.020737426, -0.037822597, -0.017844463, -0.032290965, 0.0069047646, -0.099129476, -0.004792718, -0.002004382, -0.045398016, -0.0055482923, -0.046683803, 0.019057363, -0.04309031, -0.06409597, -0.055523045, -0.02252165, 0.013671074, 0.03244552, -0.017118258, -0.07501246, -0.015934113, -0.024349216, 0.05715597, -0.027819632, -0.053864576, -0.01186773, 0.05742983, -0.050388325, -0.005809614, 0.029917926, 0.034565236, -0.008199293, -0.03518627, 0.025196387, 0.023738401, -0.0045172838, -0.049953386, -0.033262387, -0.0042250855, 0.0109193465, 0.022812199, -0.009119953, -0.024557296, -0.026358005, -0.040754016, 0.048631907, 0.058504313, 0.012611644, -0.010177845, -0.010893005, 0.04249282, 

In [6]:
DOCUMENT1 ={
    "Title": "Gato",
    "Content" : "É um animal doméstico apreciado por caçar ratos e ratazanas"
}

DOCUMENT2 = {
    "Title": "Cão",
    "Content" : "Os cães – Canis familiaris – são mamíferos quadrúpedes, pertencente ao grupo de carnívoros e da família Canidae que englobam além dos cães, os lobos, chacais, a raposa, o cachorro-do-mato, o lobo-guará e outros animais, a família é dividida em 38 espécies, sendo 6 espécies selvagens encontradas no Brasil"
}

DOCUMENT3 = {
    "Title": "Hamster",
    "Content" : "Hamster ou Cicreto são os nomes usados para diversos pequenos mamíferos roedores, pertencentes à sub-família  Cricetinae, encontrados na Ásia e África"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [17]:
df = pd.DataFrame(documents)
df.columns = ["Title", "Content"]
df

,Title,Content
0,Gato,É um animal doméstico apreciado por caçar rato...
1,Cão,Os cães – Canis familiaris – são mamíferos qua...
2,Hamster,Hamster ou Cicreto são os nomes usados para di...


In [18]:
model = "models/embedding-001"

In [19]:
def embed_fn(title, text) :
  return genai.embed_content(
    model=model,
    content=sample_text,
    title=title,
    task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [20]:
  df["Embeddings"] = df.apply(lambda row: embed_fn(row["Title"], row["Content"]), axis=1)
  df

,Title,Content,Embeddings
0,Gato,É um animal doméstico apreciado por caçar rato...,"[0.029133508, -0.00116755, -0.054318823, -0.02..."
1,Cão,Os cães – Canis familiaris – são mamíferos qua...,"[0.027891524, -0.017499419, -0.05861357, -0.03..."
2,Hamster,Hamster ou Cicreto são os nomes usados para di...,"[0.02158699, -0.03158673, -0.07927357, -0.0487..."


In [35]:
def gen_and_fetch(query, base, model) :
  gen_and_fetch_embeding = genai.embed_content(
    model=model,
    content=query,
    task_type="RETRIEVAL_QUERY")["embedding"]

  scalar_product = np.dot(np.stack(df["Embeddings"]), gen_and_fetch_embeding)

  indx = np.argmax(scalar_product)
  return df.iloc[indx]["Content"]

In [39]:
track = gen_and_fetch("O que e um Gato ?", df, model)
print(track)

Hamster ou Cicreto são os nomes usados para diversos pequenos mamíferos roedores, pertencentes à sub-família  Cricetinae, encontrados na Ásia e África
